## This code assumes the following:
1. Snowphlake package has already been installed.
2. SustaIn package has already been installed.
3. neuroCombat harmonization package has already been installed.
4. compress_pickle package has already been installed.
5. if the jupyter notebook is placed in a directory named "dir_path", the Freesurfer v7.1 volumetric data file with patients to be subtyped is placed in dir_path/inputs/
6. The trained models are stored in dir_path/trained_models/
7. dir_path/outputs/ folder exists with write permission to store the output from this analysis.

## Input file format
The volumetric input file in dir_path/inputs/ should be named pre_harmonization.csv
pre_harmonization.csv should have the following columns:
1. volumes of the regions of interest i.e., Subcortical volumes and Cortical volumes (Desikan Kiliany Atlas). Exact names of the fields are mentioned in the variable "rois" below
2. Sex: should be either 0 / 1 {0 for male, 1 for female}
3. eTIV: Total intracranial volume
4. Age: age of the patient 
5. Diagnosis (CN / SCD / MCI / AD): For a clinical diagnosis of Cognitively normal, Subjective cognitive decline, Mild cognitive impairment or AD dementia.
6. Scanner: This should be in the following format: ManufacturerModelName (FieldStrength)
    For example: Discovery MR750 (3.0) or Skyra (3.0) etc.

## ADNI reference batch 
For harmonization of the volumes in a test cohort with our training cohort, we use data of a reference batch of patients from ADNI cohort (consisting of n=257 patients). ADNI MRI and associated data should be downloaded from adni.loni.usc.edu. The RIDs of the ADNI patients needed for harmonization are in ADNI_reference_batch.csv. Please process the baseline MRI data of these patients and fill the csv file in a way that's identical to the input file as described above and place it in dir_path/inputs/ADNI_reference_batch.csv


In [ ]:
import pandas as pd 
import compress_pickle
import numpy as np
import os
from neuroCombat import neuroCombat

dir_path = os.path.dirname(os.path.realpath(__file__))
os.chdir(dir_path)

str_data = dir_path + '/inputs/'
str_result = dir_path + '/outputs/'
str_models  = dir_path + '/trained_models/'

rois = ['Left-Thalamus', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum', 'Left-Hippocampus',
 'Left-Amygdala', 'CSF', 'Left-Accumbens-area', 'Right-Thalamus', 'Right-Caudate', 'Right-Putamen',
 'Right-Pallidum', 'Right-Hippocampus', 'Right-Amygdala', 'Right-Accumbens-area', 'BrainSegVol',
 'BrainSegVolNotVent', 'TotalGrayVol', 'lh_bankssts_volume', 'lh_caudalanteriorcingulate_volume',
 'lh_caudalmiddlefrontal_volume', 'lh_cuneus_volume', 'lh_entorhinal_volume', 'lh_fusiform_volume',
 'lh_inferiorparietal_volume', 'lh_inferiortemporal_volume', 'lh_isthmuscingulate_volume', 'lh_lateraloccipital_volume',
 'lh_lateralorbitofrontal_volume', 'lh_lingual_volume', 'lh_medialorbitofrontal_volume', 'lh_middletemporal_volume',
 'lh_parahippocampal_volume', 'lh_paracentral_volume', 'lh_parsopercularis_volume', 'lh_parsorbitalis_volume',
 'lh_parstriangularis_volume', 'lh_pericalcarine_volume', 'lh_postcentral_volume', 'lh_posteriorcingulate_volume',
 'lh_precentral_volume', 'lh_precuneus_volume', 'lh_rostralanteriorcingulate_volume', 'lh_rostralmiddlefrontal_volume',
 'lh_superiorfrontal_volume', 'lh_superiorparietal_volume', 'lh_superiortemporal_volume', 'lh_supramarginal_volume',
 'lh_frontalpole_volume', 'lh_temporalpole_volume', 'lh_transversetemporal_volume', 'lh_insula_volume',
 'rh_bankssts_volume', 'rh_caudalanteriorcingulate_volume', 'rh_caudalmiddlefrontal_volume', 'rh_cuneus_volume',
 'rh_entorhinal_volume', 'rh_fusiform_volume', 'rh_inferiorparietal_volume', 'rh_inferiortemporal_volume',
 'rh_isthmuscingulate_volume', 'rh_lateraloccipital_volume', 'rh_lateralorbitofrontal_volume', 'rh_lingual_volume',
 'rh_medialorbitofrontal_volume', 'rh_middletemporal_volume', 'rh_parahippocampal_volume', 'rh_paracentral_volume',
 'rh_parsopercularis_volume', 'rh_parsorbitalis_volume', 'rh_parstriangularis_volume', 'rh_pericalcarine_volume',
 'rh_postcentral_volume', 'rh_posteriorcingulate_volume', 'rh_precentral_volume', 'rh_precuneus_volume',
 'rh_rostralanteriorcingulate_volume', 'rh_rostralmiddlefrontal_volume', 'rh_superiorfrontal_volume',
 'rh_superiorparietal_volume', 'rh_superiortemporal_volume', 'rh_supramarginal_volume', 'rh_frontalpole_volume',
 'rh_temporalpole_volume', 'rh_transversetemporal_volume', 'rh_insula_volume']


In [ ]:
# Harmonization 

Xtest = pd.read_csv(str_data+'pre_harmonization.csv', low_memory=False)
for r in rois:
    Xtest[r] = Xtest[r] / Xtest['eTIV']

In [ ]:
categorical_cols = ['Sex','Diagnosis']
batch_col = 'scanner'
continuous_cols = ['Age','eTIV']

ADNI_ref = pd.read_csv(str_data + 'ADNI_reference_batch.csv')
scanner_ref = ADNI_ref.loc[0,'scanner']

combined_dataframe = pd.concat((Xtest,ADNI_ref))
combined_dataframe = combined_dataframe.reset_index(drop=True)

covars = combined_dataframe[['scanner','Sex','Diagnosis','Age','eTIV']]

data=np.transpose(combined_dataframe[rois].values)
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col,
    ref_batch = scanner_ref,
    categorical_cols=categorical_cols,
    continuous_cols = continuous_cols,
    eb=False,parametric=True)['data']

combined_dataframe[rois] = np.transpose(data_combat)

idx_ref = combined_dataframe['scanner'] == scanner_ref
Xharmonized = combined_dataframe[~idx_ref].reset_index(drop=True)

Xharmonized.to_csv(str_result + 'full_harmonized_combat.csv', index=False)

In [ ]:
## Step 3: Combine ROIs and correct volumes for age
import regions_of_interest

data = pd.read_csv(str_result + 'full_harmonized_combat.csv')
data_mapped,BiomarkersList = regions_of_interest.map(data, rois)
biomarkers_selected = BiomarkersList
idx_cn = data_mapped['Diagnosis'] == 'CN'
data_cn = data_mapped[idx_cn].reset_index(drop=True)
factors = ['Age']
cc = compress_pickle.load(open(str_models+'/age_correction_v5_combat.gz','rb'))
data_corrected_combat= cc.predict(data_mapped)

In [ ]:
subtype_labels = ['Parieto-temporal',
                'Diffused cortical',
                'Subcortical',
                'Frontal']

data_mapped[biomarkers_selected] = data_corrected_combat[biomarkers_selected].values
T,normalize_variation = compress_pickle.load(open(str_models + 'trained_spl_model_v5_combat.gz','rb'))

biomarkers_selected = T.biomarker_labels

data = data_mapped[biomarkers_selected].values
Sp = T.predict(data,estimate_subtypes=True)

data_mapped['Snowphlake_subtype'] = Sp['subtype']
for sl,i in zip(subtype_labels,range(4)):
    idx_i = data_mapped['Snowphlake_subtype'] == i
    data_mapped.loc[idx_i,'Snowphlake_subtype'] = sl
    
data_mapped['Snowphlake_stage'] = Sp['stage']
data_mapped['Snowphlake_subtype_weight_PT'] = Sp['subtype_weights'][:,0]
data_mapped['Snowphlake_subtype_weight_DC'] = Sp['subtype_weights'][:,1]
data_mapped['Snowphlake_subtype_weight_Sc'] = Sp['subtype_weights'][:,2]
data_mapped['Snowphlake_subtype_weight_Fr'] = Sp['subtype_weights'][:,3]

In [ ]:
data_mapped[biomarkers_selected] = data_corrected_combat[biomarkers_selected].values
sustain_input,samples_sequence,samples_f,cn_mean,cn_std = compress_pickle.load(open(str_models+'trained_sustain_model_v5_combat.gz','rb'))
subtype_labels = ['Typical', 'Limbic predominant', 'Hippocampal sparing', 'Subcortical']

data = data_mapped[biomarkers_selected].values
zdata_all = data.copy()
for i in range(len(biomarkers_selected)):
    zdata_all[:,i] = -(data[:,i] - cn_mean[i])/cn_std[i]

ml_subtype_all, prob_ml_subtype_all, ml_stage_all, prob_ml_stage_all,\
    prob_subtype_all, prob_stage_all, prob_subtype_stage_all =\
    sustain_input.subtype_and_stage_individuals_newData(zdata_all,\
    samples_sequence,samples_f, data.shape[0])

data_mapped['Sustain_subtype'] = ml_subtype_all[:,0]
data_mapped['Sustain_stage'] = ml_stage_all[:,0]

idx0 = data_mapped['Sustain_stage'] == 0
data_mapped.loc[idx0,'Sustain_subtype'] = ''

for sl,i in zip(subtype_labels,range(4)):
    idx_i = data_mapped['Sustain_subtype'] == i
    data_mapped.loc[idx_i,'Sustain_subtype'] = sl

data_mapped['Sustain_subtype_probability_Ty'] = prob_subtype_all[:,0]
data_mapped['Sustain_subtype_probability_LP'] = prob_subtype_all[:,1]
data_mapped['Sustain_subtype_probability_HS'] = prob_subtype_all[:,2]
data_mapped['Sustain_subtype_probability_Sc'] = prob_subtype_all[:,3]

data_mapped.to_csv(str_result+'final_result.csv',index=False)